# 업종분류
- 기존의 업종대분류명/업종중분류명/업종소분류명을 새로운 업종 20개로 재분류
- 데이터: data_1901_2004.ftr -> 업종분류 -> data_1901_2004_classified.ftr
- 한국표준 목적별 개별소비지출분류 (COICOP-K) 참고해 18개로 분류

# '업종' col 생성
- '업종중분류명', '업종소분류명' 두 개만 읽어온 후, 메모리 최적화 (object -> category)

In [1]:
import pandas as pd
import numpy as np

data_dir = './data/feather/'
# data_1901_2004 # 90971056 rows
df = pd.read_feather(data_dir + 'data_1901_2004.ftr')
df

,기준년월,가맹점소재지1,가맹점소재지2,가맹점소재지3,업종대분류명,업종중분류명,업종소분류명,성별,연령대별,연평균소득추정,이용금액,이용건수
0,201901,강원도,강릉시,강남동,문화,학원,유아원,여성,40대,B2,1070000.0,4
1,201901,강원도,강릉시,강남동,생활,보건/위생,미용원,남성,20대,B2,48000.0,4
2,201901,강원도,강릉시,강남동,생활,연료판매,유류판매,남성,50대,B4,735000.0,4
3,201901,강원도,강릉시,강남동,생활,연료판매,주유소,남성,30대,B4,1041289.0,16
4,201901,강원도,강릉시,강남동,생활,연료판매,주유소,여성,40대,B4,426766.0,13
...,...,...,...,...,...,...,...,...,...,...,...,...
90971051,202004,충청북도,충주시,호암.직동,음식,일반음식,중국음식,여성,50대,B5,303000.0,17
90971052,202004,충청북도,충주시,호암.직동,음식,일반음식,중국음식,여성,60대 이상,B1,100000.0,5
90971053,202004,충청북도,충주시,호암.직동,음식,일반음식,중국음식,여성,60대 이상,B2,89000.0,7
90971054,202004,충청북도,충주시,호암.직동,음식,일반음식,중국음식,여성,60대 이상,B3,636000.0,29


In [8]:
import pandas as pd
import numpy as np

data_dir = '/content/drive/My Drive/영빈/data/feather/'
df = pd.read_feather(data_dir + 'data_1901_2004.ftr', columns=['업종중분류명', '업종소분류명']) # data_1901_2004 # 90971056 rows

""" 메모리 최적화 """

def reduce_mem_usage(df):
    start_mem = df.memory_usage().sum() / 1024**2
    print('Memory usage of dataframe is {:.2f} MB'.format(start_mem))

    for col in df.columns:
        col_type = df[col].dtype
        if col_type == object:
            df[col] = df[col].astype('category')

    end_mem = df.memory_usage().sum() / 1024**2
    print('Memory usage after optimization is: {:.2f} MB'.format(end_mem))
    print('Decreased by {:.1f}%'.format(100 * (start_mem - end_mem) / start_mem))

    return df

df = reduce_mem_usage(df)
df.info()

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
Memory usage of dataframe is 780.81 MB
Memory usage after optimization is: 260.28 MB
Decreased by 66.7%
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 90971056 entries, 0 to 90971055
Data columns (total 2 columns):
 #   Column  Dtype   
---  ------  -----   
 0   업종중분류명  category
 1   업종소분류명  category
dtypes: category(2)
memory usage: 260.3 MB


In [ ]:
""" 그냥 탐색 """

df[df['업종소분류명']=='주유소'] # 농업 17만 # 문화,취미 140만 # 건축,자재 52만 # 연료판매(주유소제외) 86만 # 자동차정비유지 184만 # 주유소 371만 

In [ ]:
""" 중분류에 속하는 소분류 검색 """
def search(middle):
  return sorted(list(df[df['업종중분류명'] == middle]['업종소분류명'].unique()))

search('용역서비스')

In [9]:
import warnings
warnings.filterwarnings(action='ignore')


all_data = []

# 1. 식료품

medium = ['음식료품', '건강식품']
small = [ '건강식품(회원제형태)']

df_filtered = df[df['업종중분류명'].isin(medium) | df['업종소분류명'].isin(small)]
df_filtered['업종'] = '식료품'
all_data.append(df_filtered)
count = df_filtered.shape[0]

# 2. 의류_잡화

medium = ['신변잡화', '의류']
small = ['세탁소', '신변잡화수리']

df_filtered = df[df['업종중분류명'].isin(medium) | df['업종소분류명'].isin(small)]
df_filtered['업종'] = '의류_잡화'
all_data.append(df_filtered)
count += df_filtered.shape[0]

# 3. 주거

medium = ['건축/자재']
small = ['LPG', '기타연료', '유류판매', 
         '기타용역서비스', '공공요금']

df_filtered = df[df['업종중분류명'].isin(medium) | df['업종소분류명'].isin(small)]
df_filtered['업종'] = '주거'
all_data.append(df_filtered)
count += df_filtered.shape[0]


# 4. 가정용품

medium = ['가구', '가전제품', '직물', '주방용품']
small = ['가정용품수리']

df_filtered = df[df['업종중분류명'].isin(medium) | df['업종소분류명'].isin(small)]
df_filtered['업종'] = '가정용품'
all_data.append(df_filtered)
count += df_filtered.shape[0]

# 5. 보건

medium = ['의료기관']

df_filtered = df[df['업종중분류명'].isin(medium)]
df_filtered['업종'] = '보건'
all_data.append(df_filtered)
count += df_filtered.shape[0]


# 6. 개인운송

medium = ['자동차판매', '자동차정비/유지']
small = ['주유소', '자동차서비스(회원제형태)']

df_filtered = df[df['업종중분류명'].isin(medium) | df['업종소분류명'].isin(small)]
df_filtered['업종'] = '개인운송'
all_data.append(df_filtered)
count += df_filtered.shape[0]


# 7. 여객운송

medium = ['여행업']

df_filtered = df[df['업종중분류명'].isin(medium)]
df_filtered['업종'] = '여객운송'
all_data.append(df_filtered)
count += df_filtered.shape[0]


# 8. 정보통신

medium = ['사무/통신기기']
small = ['CATV', '통신서비스', '위성방송', '소프트웨어', '이동통신요금', '정보서비스', 
         '사무통신기기수리']

df_filtered = df[df['업종중분류명'].isin(medium) | df['업종소분류명'].isin(small)]
df_filtered['업종'] = '정보통신'
all_data.append(df_filtered)
count += df_filtered.shape[0]


# 9. 오락_문화

medium = ['레져업소', '레져용품', '문화/취미', '광학제품', '서적/문구']
small = ['레져업소(회원제형태)', '레져용품수리', '서적출판(회원제형태)', '학원(회원제형태)']

df_filtered = df[df['업종중분류명'].isin(medium) | df['업종소분류명'].isin(small)]
df_filtered['업종'] = '오락_문화'
all_data.append(df_filtered)
count += df_filtered.shape[0]


# 10. 교육서비스

medium = ['학원']

df_filtered = df[df['업종중분류명'].isin(medium)]
df_filtered['업종'] = '교육서비스'
all_data.append(df_filtered)
count += df_filtered.shape[0]


# 11. 음식점

medium = ['일반음식', '단란주점(음식)', '유흥주점(음식)', '휴게']

df_filtered = df[df['업종중분류명'].isin(medium)]
df_filtered['업종'] = '음식점'
all_data.append(df_filtered)
count += df_filtered.shape[0]


# 12. 숙박서비스

medium = ['숙박업']

df_filtered = df[df['업종중분류명'].isin(medium)]
df_filtered['업종'] = '숙박서비스'
all_data.append(df_filtered)
count += df_filtered.shape[0]


# 13. 보험

medium = ['보험']

df_filtered = df[df['업종중분류명'].isin(medium)]
df_filtered['업종'] = '보험'
all_data.append(df_filtered)
count += df_filtered.shape[0]


# 14. 개인미용

medium = ['보건/위생']

df_filtered = df[df['업종중분류명'].isin(medium)]
df_filtered['업종'] = '개인미용'
all_data.append(df_filtered)
count += df_filtered.shape[0]


# 15. 기타

medium = ['기타', '농업']
small = ['기타회원제형태업소', 
         '가례서비스', '법률회계서비스', '부동산중개임대', '보관창고업', '사무서비스', '조세서비스', '종합용역', '화물운송', 
         '기타수리서비스', 
         '사무서비스(회원제형태)']

df_filtered = df[df['업종중분류명'].isin(medium) | df['업종소분류명'].isin(small)]
df_filtered['업종'] = '기타'
all_data.append(df_filtered)
count += df_filtered.shape[0]


# 16. 대형판매

small = ['대형할인점', '면세점', '일반백화점', 
         '농협하나로클럽']

df_filtered = df[df['업종소분류명'].isin(small)]
df_filtered['업종'] = '대형판매'
all_data.append(df_filtered)
count += df_filtered.shape[0]


# 17. 종합소매

small = ['편의점', '기타유통업', '농축수산가공품', '복지매점', '상품권', '슈퍼마켓', '연쇄점',
          '구내매점', '기타비영리유통', '농축협직영매장']

df_filtered = df[df['업종소분류명'].isin(small)]
df_filtered['업종'] = '종합소매'
all_data.append(df_filtered)
count += df_filtered.shape[0]


# 18. 전자상거래

small = ['인터넷Mall', '인터넷PG', '인터넷종합Mall', '전자상거래상품권', '통신판매', '홈쇼핑', 'PG상품권']

df_filtered = df[df['업종소분류명'].isin(small)]
df_filtered['업종'] = '전자상거래'
all_data.append(df_filtered)
count += df_filtered.shape[0]

print('total rows: ', count)

total rows:  90971056


In [10]:
combined = pd.concat(all_data, axis=0, ignore_index=False) 
combined

,업종중분류명,업종소분류명,업종
22,음식료품,농축수산품,식료품
51,음식료품,기타음료식품,식료품
52,음식료품,기타음료식품,식료품
53,음식료품,기타음료식품,식료품
115,음식료품,제과점,식료품
...,...,...,...
90946934,유통업영리,인터넷종합Mall,전자상거래
90946935,유통업영리,인터넷종합Mall,전자상거래
90953928,유통업영리,인터넷Mall,전자상거래
90953929,유통업영리,인터넷Mall,전자상거래


# 기존 데이터에 '업종' col 붙이기

In [11]:
df = pd.read_feather(data_dir + 'data_1901_2004.ftr', 
                     columns = ['기준년월', '가맹점소재지1', '가맹점소재지2',
                                 '성별', '연령대별', '연평균소득추정', '이용금액', '이용건수'])

new_df = pd.DataFrame(combined['업종']).astype('category')

""" index에 맞춰서 붙이기 """
df = df.join(new_df, how = 'left')

df = reduce_mem_usage(df)
df.info()

Memory usage of dataframe is 1995.42 MB
Memory usage after optimization is: 1995.42 MB
Decreased by 0.0%
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 90971056 entries, 0 to 90971055
Data columns (total 9 columns):
 #   Column   Dtype   
---  ------   -----   
 0   기준년월     int32   
 1   가맹점소재지1  category
 2   가맹점소재지2  category
 3   성별       category
 4   연령대별     category
 5   연평균소득추정  category
 6   이용금액     float64 
 7   이용건수     int32   
 8   업종       category
dtypes: category(6), float64(1), int32(2)
memory usage: 1.9 GB


# 데이터 저장

In [12]:
data_dir = '/content/drive/My Drive/영빈/data/'
df.to_feather(data_dir + 'data_1901_2004_classified.ftr')